## DSCI 100 Group Project Proposal ##

By: Alessandra Lu, Ruby Suo, Han Wei, Alan Wu

#### Introduction ####

**Question:** 

#### Preliminary Exploratory Data Analysis ####

In [5]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
library(dbplyr)

In [9]:
url <- "https://raw.githubusercontent.com/alan-w-u/dsci-100-group-project/main/student-mat.csv"
data_raw <- read_delim(url, delim = ";")
data_raw
data_tidied <- pivot_longer(data_raw, cols = famrel:health, names_to = "factor", values_to = "actual")
data_tidied

Rows: 395 Columns: 33
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (17): school, sex, address, famsize, Pstatus, Mjob, Fjob, reason, guardi...
dbl (16): age, Medu, Fedu, traveltime, studytime, failures, famrel, freetime...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GP,F,18,U,GT3,A,4,4,at_home,teacher,⋯,4,3,4,1,1,3,6,5,6,6
GP,F,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,4,5,5,6
GP,F,15,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,10,7,8,10
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
MS,M,21,R,GT3,T,1,1,other,other,⋯,5,5,3,3,3,3,3,10,8,7
MS,M,18,R,LE3,T,3,2,services,other,⋯,4,4,1,3,4,5,0,11,12,10
MS,M,19,U,LE3,T,1,1,other,at_home,⋯,3,2,3,3,3,5,5,8,9,9


school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,nursery,higher,internet,romantic,absences,G1,G2,G3,factor,actual
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
GP,F,18,U,GT3,A,4,4,at_home,teacher,⋯,yes,yes,no,no,6,5,6,6,famrel,4
GP,F,18,U,GT3,A,4,4,at_home,teacher,⋯,yes,yes,no,no,6,5,6,6,freetime,3
GP,F,18,U,GT3,A,4,4,at_home,teacher,⋯,yes,yes,no,no,6,5,6,6,goout,4
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
MS,M,19,U,LE3,T,1,1,other,at_home,⋯,yes,yes,yes,no,5,8,9,9,Dalc,3
MS,M,19,U,LE3,T,1,1,other,at_home,⋯,yes,yes,yes,no,5,8,9,9,Walc,3
MS,M,19,U,LE3,T,1,1,other,at_home,⋯,yes,yes,yes,no,5,8,9,9,health,5


#### Methods ####

#### Expected Outcomes and Significance ####